In [1]:
#인터넷 리퀘스트 계열
import requests
import urllib.request
#시간 계열
import datetime
import ntplib
from time import ctime
#파싱 계열
import xml.etree.ElementTree as et
#데이터베이스 계열
import pandas as pd
#Get Local Time from Time Server
print("한 시간전 기상 데이터를 받아옵니다.")
timeServer = 'time.windows.com'
c = ntplib.NTPClient()
response = c.request(timeServer, version=3)
now=ctime(response.tx_time)
now=datetime.datetime.strptime(now, '%a %b  %d %H:%M:%S %Y')
year=now.strftime("%Y") 
month=now.strftime("%m")
day=now.strftime("%d")
hour=now.strftime("%H")
minute=now.strftime("%M")
second=now.strftime("%S")
#필수 Parameter 설정
base_date=year+month+day
temp_h=int(hour)
temp_h-=1;
if temp_h<10 :
    hour='0'+str(temp_h)
else : hour=str(temp_h)
base_time=hour+'00'
print(base_date+base_time)
#지역정보 : kor 1168058000 서울특별시 강남구 삼성1동 nx=61 ny=125 (경기고등학교)
#Get Weather Info
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
params ={'serviceKey' : 'DyvMF7VeSRafi8aP+n1F7Ae1dXIbqVGeUuPhMxUCBeTh40qeBxY05uAzqAH0FOwX+j0TZG2lGKxMagGJ6I0NAQ==',
         'pageNo' : '1',
         'numOfRows' : '1000',
         'dataType' : 'XML',
         'base_date' : base_date,
         #'base_date' : '20220801',
         'base_time' : base_time,
         #'base_time' : '1100',
         'nx' : '61',
         'ny' : '125' }

response = requests.get(url, params=params)
str_response=response.content.decode('utf-8')
print(type(str_response))
#Write File
vcs_filename=year
f = open(vcs_filename+".xml", 'w', encoding='UTF-8')
f.write(str_response)
f.close()
#Initial Database
df=pd.DataFrame({"날짜":[],
                 "기온":[],
                 "강수량":[]})
#db = pd.read_excel('WeatherReport/database.xlsx')
#print(db)
#db.to_csv('WeatherReport/database.xlsx')
#db
#Parse File
file = et.parse(vcs_filename+".xml")
root = file.getroot()
data = root[1][1]
result = open("WeatherReport/"+vcs_filename+".csv", 'a', encoding='UTF-8')
print()
result.write("\nDate/time, "+vcs_filename+", ")
cnt = 0
base_date_printed = 0
base_time_printed = 0
current_category='0'
for child in data:
    cnt = 0
    for value in child:
        print(value.tag+value.text) #only for debugging
        if value.tag == 'baseDate' and base_date_printed == 0:
            result.write("정보갱신날짜, "+value.text+", ")
            base_date_printed = 1
        if value.tag == 'baseTime' and base_time_printed == 0:
            result.write("정보갱신시간, "+value.text+", ")
            base_time_printed = 1
        if value.text == 'T1H' : 
            result.write("기온, ")
            current_category=value.text
        if value.text == 'RN1' : 
            result.write("1시간 강수량, ")
            current_category=value.text
        if value.text == 'UUU' : 
            result.write("동서바람성분, ")
            current_category=value.text
        if value.text == 'VVV' : 
            result.write("남북바람성분, ")
            current_category=value.text
        if value.text == 'REH' : 
            result.write("습도, ")
            current_category=value.text
        if value.text == 'PTY' : 
            result.write("강수형태, ")
            current_category=value.text
        if value.text == 'VEC' : 
            result.write("풍향, ")
            current_category=value.text
        if value.text == 'WSD' : 
            result.write("풍속, ")
            current_category=value.text
        end_sequence = 0
        if value.tag == 'obsrValue' and  current_category == 'PTY' :
            if value.text == '0' : result.write( '맑음')
            if value.text == '1' : result.write( '비')
            if value.text == '2' : result.write( '비/눈')
            if value.text == '3' : result.write( '눈')
            if value.text == '5' : result.write( '빗방울')
            if value.text == '6' : result.write( '빗방울눈날림')
            if value.text == '7' : result.write( '눈날림')
            end_sequence = 1
        if value.tag == 'obsrValue' and end_sequence == 0 : result.write(value.text)
    result.write(", ")                                                                                                               
result.close()
df = pd.read_csv('2022.csv', encoding='utf-8')


한 시간전 기상 데이터를 받아옵니다.
202208142200
<class 'str'>

baseDate20220814
baseTime2200
categoryPTY
nx61
ny125
obsrValue0
baseDate20220814
baseTime2200
categoryREH
nx61
ny125
obsrValue84
baseDate20220814
baseTime2200
categoryRN1
nx61
ny125
obsrValue0
baseDate20220814
baseTime2200
categoryT1H
nx61
ny125
obsrValue27.6
baseDate20220814
baseTime2200
categoryUUU
nx61
ny125
obsrValue-0.4
baseDate20220814
baseTime2200
categoryVEC
nx61
ny125
obsrValue160
baseDate20220814
baseTime2200
categoryVVV
nx61
ny125
obsrValue1.4
baseDate20220814
baseTime2200
categoryWSD
nx61
ny125
obsrValue1.5


In [20]:
#time request using ntp
#datetime 함수 이용 시간 가공
import datetime
import ntplib
from time import ctime

timeServer = 'time.windows.com'
c = ntplib.NTPClient()
response = c.request(timeServer, version=3)
now=ctime(response.tx_time)
now=datetime.datetime.strptime(now, '%a %b  %d %H:%M:%S %Y')
year=now.strftime("%Y")
month=now.strftime("%m")
day=now.strftime("%d")
hour=now.strftime("%H")
minute=now.strftime("%M")
second=now.strftime("%S")
print(type(year))
print(year,"/",month,"/",day," ",hour,":",minute,":",second,sep="")

<class 'str'>
2022/08/05 11:16:02


In [3]:
#time request using urllib
import urllib.request
month = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', \
    'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

url = 'http://www.daum.net'
date = urllib.request.urlopen(url).headers['Date'][5:-4]
d, m, y, hour, min, sec = date[:2], month[date[3:6]], date[7:11], date[12:14], date[15:17], date[18:]
print(f'[{url}]의 서버시간\n{y}년 {m}월 {d}일 {hour}시 {min}분 {sec}초')
#Archived

[http://www.daum.net]의 서버시간
2022년 07월 09일 11시 19분 42초


In [13]:
#Archived
from datetime import datetime

current_time = datetime.now()
print(current_time)

2022-07-08 10:55:56.572070


In [6]:
import requests
import time
import xml.etree.ElementTree as et

def main():
    local = time.localtime(time.time()-3600)

    legend = ['강수형태','습도','1시간 강수량','기온','동서바람성분','풍향','남북바람성분','풍속']

    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
    params = {'serviceKey': 'loGEsi8RLLPDoscN/+N/FQs4Gck/La0uzw51zooTczK3aWcmK6mvoHJ8tfF1uCtnKl2OdfmXYnz1QtEauhwWPA==','pageNo': '1', 'numOfRows': '1000', 'dataType': 'XML','base_date': str(local.tm_year).zfill(2) + str(local.tm_mon).zfill(2) + str(local.tm_mday).zfill(2),'base_time': str(local.tm_hour).zfill(2) + str(local.tm_min).zfill(2), 'nx': '61', 'ny': '126'}

    response = requests.get(url, params=params)
    text = response.text
    fname = str(local.tm_year).zfill(2)+str(local.tm_mon).zfill(2)+str(local.tm_mday).zfill(2)+str(local.tm_hour).zfill(2)+str(local.tm_min).zfill(2)
    f = open("report"+".xml", 'w', encoding='UTF-8')
    f.write(text)
    f.close()

    file = et.parse("report"+".xml")
    root = file.getroot()

    try:
        data = root[1][1]
    except:
        print("Error in data")
        exit()

    result = open("results/"+fname+".txt", 'w', encoding='UTF-8')

    i = 0
    for child in data:
        result.write(legend[i]+'\n')
        print(legend[i])
        print(child[5].text)
        result.write(child[5].text + ' ' + "\n")
        result.write("\n")
        print()
        i=i+1

    result.close()

if __name__ == '__main__':
    main()

강수형태
0

습도
75

1시간 강수량
0

기온
29.2

동서바람성분
1.2

풍향
275

남북바람성분
0

풍속
1.2



In [16]:
db

,Date,Time,Humid,Rainfall,Temp


In [5]:
#인터넷 리퀘스트 계열
import requests
import urllib.request
#시간 계열
import datetime
import ntplib
from time import ctime
#파싱 계열
import xml.etree.ElementTree as et
#데이터베이스 계열
import pandas as pd
#Get Local Time from Time Server
print("한 시간전 기상 데이터를 받아옵니다.")
timeServer = 'time.windows.com'
c = ntplib.NTPClient()
response = c.request(timeServer, version=3)
now=ctime(response.tx_time)
now=datetime.datetime.strptime(now, '%a %b  %d %H:%M:%S %Y')
year=now.strftime("%Y")
month=now.strftime("%m")
day=now.strftime("%d")
hour=now.strftime("%H")
minute=now.strftime("%M")
second=now.strftime("%S")
#필수 Parameter 설정
base_date=year+month+day
temp_h=int(hour)
temp_h-=1;
if temp_h<10 :
    hour='0'+str(temp_h)
else : hour=str(temp_h)
base_time=hour+'00'
print(base_date+base_time)
#지역정보 : kor 1168058000 서울특별시 강남구 삼성1동 nx=61 ny=125 (경기고등학교)
#Get Weather Info
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
params ={'serviceKey' : 'DyvMF7VeSRafi8aP+n1F7Ae1dXIbqVGeUuPhMxUCBeTh40qeBxY05uAzqAH0FOwX+j0TZG2lGKxMagGJ6I0NAQ==',
         'pageNo' : '1',
         'numOfRows' : '1000',
         'dataType' : 'XML',
         'base_date' : base_date,
         #'base_date' : '20220801',
         'base_time' : base_time,
         #'base_time' : '1100',
         'nx' : '61',
         'ny' : '125' }

response = requests.get(url, params=params)
str_response=response.content.decode('utf-8')
print(type(str_response))
#Write File
vcs_filename=year+month+day+'-'+hour+'00'
f = open(vcs_filename+".xml", 'w', encoding='UTF-8')
f.write(str_response)
f.close()
#Initial Database
df=pd.DataFrame({"날짜":[],
                 "기온":[],
                 "강수량":[]})
#db = pd.read_excel('WeatherReport/database.xlsx')
#print(db)
#db.to_csv('WeatherReport/database.xlsx')
#db
#Parse File
file = et.parse(vcs_filename+".xml")
root = file.getroot()
data = root[1][1]
print(data)
result = open("WeatherReport/report.csv", 'a', encoding='UTF-8')
print()
result.write("Date/time, "+vcs_filename+", ")
cnt = 0
base_date_printed = 0
base_time_printed = 0
current_category='0'
for child in data:
    cnt = 0
    for value in child:
        #print(value.tag+value.text) #only for debugging
        if value.tag == 'baseDate' and base_date_printed == 0:
            result.write("정보갱신날짜, "+value.text+", ")
            base_date_printed = 1
        if value.tag == 'baseTime' and base_time_printed == 0:
            result.write("정보갱신시간, "+value.text+", ")
            base_time_printed = 1
        if value.text == 'T1H' : 
            result.write("기온, ")
            current_category=value.text
        if value.text == 'RN1' : 
            result.write("1시간 강수량, ")
            current_category=value.text
        if value.text == 'REH' : 
            result.write("습도, ")
            current_category=value.text
        if value.text == 'PTY' : 
            result.write("강수형태, ")
            current_category=value.text
        end_sequence = 0
        if value.tag == 'obsrValue' and  current_category == 'PTY' :
            if value.text == '0' : result.write(', '+ '맑음')
            if value.text == '1' : result.write(', '+ '비')
            if value.text == '2' : result.write(', '+ '비/눈')
            if value.text == '3' : result.write(', '+ '눈')
            if value.text == '5' : result.write(', '+ '빗방울')
            if value.text == '6' : result.write(', '+ '빗방울눈날림')
            if value.text == '7' : result.write(', '+ '눈날림')
            end_sequence = 1
        if value.tag == 'obsrValue' and end_sequence == 0 : result.write(', '+value.text)
    result.write(", ")
    
result.close()

한 시간전 기상 데이터를 받아옵니다.
202208050900
<class 'str'>
<Element 'items' at 0x000002922C122D60>



한 시간전 기상 데이터를 받아옵니다.
202208050900
<class 'str'>
<Element 'items' at 0x000002922C137180>

obsrValue
20220805
0900
PTY
61
125
0
obsrValue
20220805
0900
REH
61
125
78
obsrValue
20220805
0900
RN1
61
125
0
obsrValue
20220805
0900
T1H
61
125
29.5
obsrValue
20220805
0900
UUU
61
125
-0.8
obsrValue
20220805
0900
VEC
61
125
145
obsrValue
20220805
0900
VVV
61
125
1.3
obsrValue
20220805
0900
WSD
61
125
1.6
